In [33]:
import sqlite3
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [34]:
path = ''
db_name= 'sub.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    #　SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブル作成
    sql = 'CREATE TABLE google_repo ( repo_name TEXT, lang_name TEXT, star INT);'

    # SQLの実行
    cur.execute(sql)

except sqlite3.Error as e:
    print(f"エラーが発生しました: {e}")

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: table google_repo already exists


In [35]:
DB_NAME = "sub.db"
BASE_URL = "https://github.com/google"

In [36]:
def convert_star_count(text):
    text = text.lower().replace(",", "").strip()
    if "k" in text:
        return int(float(text.replace("k", "")) * 1000)
    elif "m" in text:
        return int(float(text.replace("m", "")) * 1_000_000)
    else:
        try:
            return int(text)
        except:
            return 0


In [ ]:
def scrape_google_repos():
    repos = []

    # Chromeをヘッドレスで起動
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    # Googleのリポジトリ一覧ページ
    BASE_URL = "https://github.com/google?tab=repositories"
    driver.get(BASE_URL)
    time.sleep(3)  

    soup = BeautifulSoup(driver.page_source, "html.parser")
    repo_items = soup.select("li.public")  

    print("取得したリポジトリ数:", len(repo_items))

    for i, repo in enumerate(repo_items, start=1):
        # リポジトリ名
        repo_name_tag = repo.select_one("h3 a")
        repo_name = repo_name_tag.text.strip() if repo_name_tag else "Unknown"

        # 言語
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']")
        lang_name = lang_tag.text.strip() if lang_tag else "Unknown"

        # スター
        star_tag = repo.select_one("a[href$='/stargazers']")
        star = convert_star_count(star_tag.text.strip()) if star_tag else 0

        repos.append((repo_name, lang_name, star))
        print(f"[{i}] {repo_name} / {lang_name} / {star}")

    driver.quit()
    return repos

In [38]:
def save_to_db(repos):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    cur.executemany(
        "INSERT INTO google_repo (repo_name, lang_name, star) VALUES (?, ?, ?)",
        repos
    )
    conn.commit()
    conn.close()
    print("スクレイピング結果を google_repo テーブルに保存しました！")

In [39]:
data = scrape_google_repos()
save_to_db(data)


取得したリポジトリ数: 0
スクレイピング結果を google_repo テーブルに保存しました！


In [40]:
conn = sqlite3.connect(DB_NAME)
cur = conn.cursor()
cur.execute("SELECT * FROM google_repo")
rows = cur.fetchall()
for r in rows:
    print(r)
conn.close()